In [1]:
import sys
import boto3
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

spark_ctx = SparkContext()
glue_ctx = GlueContext(spark_ctx)
spark_ses = glue_ctx.spark_session

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/home/glue_user/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getO

In [3]:
aws_region = "us-east-1"
boto_ses = boto3.session.Session(region_name=aws_region)
response = boto_ses.client("sts").get_caller_identity()
aws_account_id = response["Account"]
bucket = f"{aws_account_id}-{aws_region}-data"

s3_client = boto_ses.client("s3")

tracker_key = "projects/dynamodb_to_datalake/tracker.txt"
try:
    response = s3_client.get_object(Bucket=bucket, key=tracker_key)
    last_processed_key = response["Body"].read().decode("utf-8")
except Exception as e:
    last_processed_key = "projects/dynamodb_to_datalake/dynamodb_stream/2023-01-01-00-00-00-000000.json"

s3uri_list = list()
key_list = list()

paginator = s3_client.get_paginator("list_objects_v2")
for response in paginator.paginate(
    Bucket=bucket,
    Prefix="projects/dynamodb_to_datalake/dynamodb_stream/",
    PaginationConfig=dict(
        PageSize=1000,
    )
):
    for content in response["Contents"]:
        key = content["Key"]
        if key > last_processed_key:
            s3uri_list.append("s3://{}/{}".format(bucket, key))
            key_list.append(key)

if len(s3uri_list):
    for s3uri in s3uri_list:
        print(s3uri)

    gdf = glue_ctx.create_dynamic_frame.from_options(
        connection_type="s3",
        connection_options={
            "paths": s3uri_list,
        },
        format="json",
        format_options={
            "multiline": True,
        },
    )
    pdf = gdf.toDF()

{'UserId': 'AIDA3X7AXU2QMU6KWRZ33', 'Account': '807388292768', 'Arn': 'arn:aws:iam::807388292768:user/sanhe', 'ResponseMetadata': {'RequestId': '3ef2a86f-1889-478e-8ca8-1366b010f0f1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '3ef2a86f-1889-478e-8ca8-1366b010f0f1', 'content-type': 'text/xml', 'content-length': '402', 'date': 'Tue, 25 Jul 2023 04:02:56 GMT'}, 'RetryAttempts': 0}}
s3://807388292768-us-east-1-data/projects/dynamodb_to_datalake/dynamodb_stream/2023-07-25-03-23-24-952068.json
s3://807388292768-us-east-1-data/projects/dynamodb_to_datalake/dynamodb_stream/2023-07-25-03-23-25-267750.json
s3://807388292768-us-east-1-data/projects/dynamodb_to_datalake/dynamodb_stream/2023-07-25-03-23-25-451330.json
s3://807388292768-us-east-1-data/projects/dynamodb_to_datalake/dynamodb_stream/2023-07-25-03-23-25-808103.json
s3://807388292768-us-east-1-data/projects/dynamodb_to_datalake/dynamodb_stream/2023-07-25-03-23-25-889076.json
s3://807388292768-us-east-1-data/projects/dyna

In [4]:
pdf.show()

+---+--------------------+------+--------------------+
| id|               title|rating|   rating_updated_at|
+---+--------------------+------+--------------------+
|546|Analysis expect i...|    67|2023-07-25T03:18:...|
|709|Measure strategy ...|    89|2023-07-25T03:18:...|
|543|Happy nothing rel...|    69|2023-07-25T03:18:...|
|783|It growth white b...|    48|2023-07-25T03:18:...|
|841|Service bring on ...|    39|2023-07-25T03:18:...|
| 93|Coach ball a such...|     5|2023-07-25T03:18:...|
|975|Card or response ...|    85|2023-07-25T03:18:...|
|443|Parent out wait s...|    21|2023-07-25T03:18:...|
|143|None start throug...|    28|2023-07-25T03:18:...|
|457|Government compan...|    16|2023-07-25T03:18:...|
|580|Within interview ...|    34|2023-07-25T03:18:...|
|247|Effort claim summ...|    70|2023-07-25T03:18:...|
|427|    Lead base store.|    85|2023-07-25T03:18:...|
|494|Foot partner came...|    95|2023-07-25T03:18:...|
|912|  Side create coach.|    81|2023-07-25T03:18:...|
|131|Prepa

In [22]:
pdf.show()

+---+--------------------+------+--------------------+
| id|               title|rating|   rating_updated_at|
+---+--------------------+------+--------------------+
|546|Analysis expect i...|    67|2023-07-25T03:18:...|
|709|Measure strategy ...|    89|2023-07-25T03:18:...|
|543|Happy nothing rel...|    69|2023-07-25T03:18:...|
|783|It growth white b...|    48|2023-07-25T03:18:...|
|841|Service bring on ...|    39|2023-07-25T03:18:...|
| 93|Coach ball a such...|     5|2023-07-25T03:18:...|
|975|Card or response ...|    85|2023-07-25T03:18:...|
|443|Parent out wait s...|    21|2023-07-25T03:18:...|
|143|None start throug...|    28|2023-07-25T03:18:...|
|457|Government compan...|    16|2023-07-25T03:18:...|
|580|Within interview ...|    34|2023-07-25T03:18:...|
|247|Effort claim summ...|    70|2023-07-25T03:18:...|
|427|    Lead base store.|    85|2023-07-25T03:18:...|
|494|Foot partner came...|    95|2023-07-25T03:18:...|
|912|  Side create coach.|    81|2023-07-25T03:18:...|
|131|Prepa

In [27]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, lit

In [24]:
window_spec = Window.partitionBy(["id"]).orderBy("rating_updated_at")
pdf_new = pdf.withColumn(
    "row_number", 
    row_number().over(window_spec)
)
pdf_new.show()

+---+--------------------+------+--------------------+----------+
| id|               title|rating|   rating_updated_at|row_number|
+---+--------------------+------+--------------------+----------+
|  1|Center skill part...|    49|2023-07-25T03:19:...|         1|
|  1|Center skill part...|    72|2023-07-25T03:19:...|         2|
|  2|Large direction h...|    33|2023-07-25T03:19:...|         1|
|  3|Theory box weight...|     5|2023-07-25T03:19:...|         1|
|  6|First push before...|    67|2023-07-25T03:20:...|         1|
|  8|Above analysis tr...|    92|2023-07-25T03:19:...|         1|
|  8|Above analysis tr...|    16|2023-07-25T03:20:...|         2|
|  9|Citizen second ex...|    49|2023-07-25T03:20:...|         1|
|  9|Citizen second ex...|    20|2023-07-25T03:20:...|         2|
| 12|Imagine parent on...|    89|2023-07-25T03:19:...|         1|
| 12|Imagine parent on...|    75|2023-07-25T03:19:...|         2|
| 15|Always themselves...|    27|2023-07-25T03:18:...|         1|
| 15|Alway

In [31]:
pdf_new.select(
    "id", "title", "rating", "rating_updated_at",
).where(pdf_new.row_number == 1).withColumn(
    "year",
    lit(2023),
).show()

+---+--------------------+------+--------------------+----+
| id|               title|rating|   rating_updated_at|year|
+---+--------------------+------+--------------------+----+
|  1|Center skill part...|    49|2023-07-25T03:19:...|2023|
|  2|Large direction h...|    33|2023-07-25T03:19:...|2023|
|  3|Theory box weight...|     5|2023-07-25T03:19:...|2023|
|  6|First push before...|    67|2023-07-25T03:20:...|2023|
|  8|Above analysis tr...|    92|2023-07-25T03:19:...|2023|
|  9|Citizen second ex...|    49|2023-07-25T03:20:...|2023|
| 12|Imagine parent on...|    89|2023-07-25T03:19:...|2023|
| 15|Always themselves...|    27|2023-07-25T03:18:...|2023|
| 16|New performance p...|    33|2023-07-25T03:18:...|2023|
| 17|Issue own represe...|    71|2023-07-25T03:19:...|2023|
| 19|Republican happy ...|    26|2023-07-25T03:20:...|2023|
| 20|Difference rememb...|    95|2023-07-25T03:19:...|2023|
| 22|Party should neve...|    56|2023-07-25T03:20:...|2023|
| 23|Purpose service b...|    52|2023-07

In [16]:
sql = """
SELECT
    *
FROM 
    df_movie
LIMIT 5
""".strip()

pdf_filtered = spark_ses.sql(sql)
pdf_filtered.show()

AnalysisException: Table or view not found: df_movie; line 4 pos 4;
'GlobalLimit 5
+- 'LocalLimit 5
   +- 'Project [*]
      +- 'UnresolvedRelation [df_movie], [], false
